# Day 6: Transfer Learning and Pretrained Models

**Time:** 3-4 hours

**Mathematical Prerequisites:**
- Understanding of CNNs (from Day 2-3)
- Feature representations and learned hierarchies
- Optimization concepts (learning rates, gradient flow)

---

## Objectives

Today we explore **transfer learning**: using knowledge learned from one task (ImageNet classification) to solve another task (CIFAR-10 classification).

Key concepts:
1. Why transfer learning works (feature hierarchy)
2. Feature extraction vs fine-tuning
3. Working with pretrained models (ResNet, VGG, MobileNet)
4. Layer freezing and selective fine-tuning
5. Comparison of different strategies

**Goal:** Achieve >85% accuracy on CIFAR-10 using transfer learning

---

## Part 1: Theory - Why Transfer Learning Works

### 1.1 Feature Hierarchy in CNNs

CNNs learn hierarchical features:
- **Early layers:** Low-level features (edges, textures, colors)
- **Middle layers:** Mid-level features (patterns, shapes, object parts)
- **Late layers:** High-level features (specific to the task)

**Key Insight:** Early and middle layers learn **general** features that transfer well across tasks. Only the final layers are task-specific.

### 1.2 Transfer Learning Strategies

**1. Feature Extraction (Frozen Backbone)**
- Freeze all pretrained layers
- Only train the new classification head
- Fast, requires less data
- Use when: Small dataset, similar to ImageNet

**2. Fine-Tuning (Unfrozen Layers)**
- Unfreeze some/all pretrained layers
- Train with small learning rate
- Slower, requires more data
- Use when: Moderate dataset, somewhat different from ImageNet

**3. From Scratch**
- Initialize randomly, train everything
- Slowest, requires large dataset
- Use when: Very large dataset, very different from ImageNet

### 1.3 Learning Rate Strategy for Fine-Tuning

When fine-tuning, use **discriminative learning rates**:
- **Early layers:** Very small LR (e.g., 1e-5) - already learned good features
- **Middle layers:** Medium LR (e.g., 1e-4)
- **Final layers:** Larger LR (e.g., 1e-3) - need to adapt to new task

This prevents catastrophic forgetting of pretrained features.

### 1.4 ImageNet vs CIFAR-10

**ImageNet:**
- 1000 classes
- 224×224 images
- 1.2M training images

**CIFAR-10:**
- 10 classes (airplane, car, bird, cat, deer, dog, frog, horse, ship, truck)
- 32×32 images (much smaller!)
- 50K training images

We'll need to handle the resolution difference.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import time
from tqdm import tqdm

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 2: Load CIFAR-10 Dataset

CIFAR-10 contains 32×32 color images in 10 classes.

In [ ]:
# CIFAR-10 classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck']

# For pretrained models, we need ImageNet normalization
# ImageNet mean and std
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# Transforms for training (with augmentation)
transform_train = transforms.Compose([
    transforms.Resize(224),  # Resize to ImageNet size
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

# Transforms for testing (no augmentation)
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

# Load datasets
print("Loading CIFAR-10 dataset...")
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Number of classes: {len(classes)}")
print(f"Classes: {classes}")

### Visualize CIFAR-10 Samples

In [ ]:
# Load original CIFAR-10 for visualization (without normalization)
train_dataset_viz = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, 
    transform=transforms.ToTensor()
)

# Visualize samples from each class
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i, class_name in enumerate(classes):
    # Find first sample of this class
    for img, label in train_dataset_viz:
        if label == i:
            axes[i].imshow(img.permute(1, 2, 0))
            axes[i].set_title(class_name)
            axes[i].axis('off')
            break

plt.suptitle('CIFAR-10 Sample Images (32×32)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Show batch of images
dataiter = iter(train_loader)
images, labels = next(dataiter)

print(f"\nBatch shape: {images.shape}")
print(f"Labels shape: {labels.shape}")
print(f"After preprocessing, images are: {images.shape[2]}×{images.shape[3]}")

## Part 3: Strategy 1 - Feature Extraction (Frozen Backbone)

We'll use ResNet18 pretrained on ImageNet and only train the final classification layer.

### 3.1 Understanding ResNet18 Architecture

ResNet18 structure:
```
Input (3×224×224)
  ↓
conv1 (7×7, stride 2) → 64×112×112
  ↓
maxpool → 64×56×56
  ↓
layer1 (ResBlocks) → 64×56×56
  ↓
layer2 (ResBlocks) → 128×28×28
  ↓
layer3 (ResBlocks) → 256×14×14
  ↓
layer4 (ResBlocks) → 512×7×7
  ↓
avgpool → 512×1×1
  ↓
fc → 1000 classes (ImageNet)
```

We'll replace the final `fc` layer with our own for 10 classes.

In [ ]:
def create_feature_extractor():
    """Create ResNet18 with frozen backbone for feature extraction."""
    # Load pretrained ResNet18
    model = models.resnet18(pretrained=True)
    
    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace final fully connected layer
    # ResNet18 has 512 features before the fc layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)  # 10 classes for CIFAR-10
    
    return model, num_features

# Create model
feature_extractor, num_features = create_feature_extractor()
feature_extractor = feature_extractor.to(device)

print(f"Number of features before fc layer: {num_features}")
print(f"\nModel structure:")
print(feature_extractor)

# Count trainable parameters
total_params = sum(p.numel() for p in feature_extractor.parameters())
trainable_params = sum(p.numel() for p in feature_extractor.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")

### 3.2 Training Helper Functions

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': running_loss / (pbar.n + 1),
            'acc': 100. * correct / total
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


def evaluate(model, test_loader, criterion, device):
    """Evaluate on test set."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, np.array(all_preds), np.array(all_labels)

### 3.3 Train Feature Extractor

In [ ]:
# Setup training
criterion = nn.CrossEntropyLoss()
optimizer_fe = optim.Adam(feature_extractor.fc.parameters(), lr=0.001)
num_epochs = 10

# Track metrics
fe_train_losses = []
fe_train_accs = []
fe_test_losses = []
fe_test_accs = []

print("Training Feature Extractor (Frozen Backbone)...\n")
start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, train_acc = train_one_epoch(
        feature_extractor, train_loader, criterion, optimizer_fe, device
    )
    fe_train_losses.append(train_loss)
    fe_train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, _, _ = evaluate(
        feature_extractor, test_loader, criterion, device
    )
    fe_test_losses.append(test_loss)
    fe_test_accs.append(test_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

fe_time = time.time() - start_time
print(f"\nFeature extraction training completed in {fe_time:.2f} seconds")
print(f"Best test accuracy: {max(fe_test_accs):.2f}%")

## Part 4: Strategy 2 - Fine-Tuning

Now we'll unfreeze some layers and fine-tune with a smaller learning rate.

### 4.1 Selective Layer Unfreezing

We'll unfreeze the last ResNet block (layer4) and the fc layer.

In [ ]:
def create_finetuned_model():
    """Create ResNet18 with selective unfreezing for fine-tuning."""
    model = models.resnet18(pretrained=True)
    
    # Freeze early layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Unfreeze layer4 (last ResNet block)
    for param in model.layer4.parameters():
        param.requires_grad = True
    
    # Replace and unfreeze fc layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)
    
    return model

# Create model
finetuned_model = create_finetuned_model()
finetuned_model = finetuned_model.to(device)

# Count parameters
total_params = sum(p.numel() for p in finetuned_model.parameters())
trainable_params = sum(p.numel() for p in finetuned_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")

# Show which layers are trainable
print("\nTrainable layers:")
for name, param in finetuned_model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.numel():,} parameters")

### 4.2 Discriminative Learning Rates

We'll use different learning rates for different parts of the network:
- layer4: 1e-4 (smaller LR for pretrained features)
- fc: 1e-3 (larger LR for new classifier)

In [ ]:
# Optimizer with different learning rates per layer group
optimizer_ft = optim.Adam([
    {'params': finetuned_model.layer4.parameters(), 'lr': 1e-4},
    {'params': finetuned_model.fc.parameters(), 'lr': 1e-3}
])

# Learning rate scheduler
scheduler_ft = optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

print("Optimizer configuration:")
for i, param_group in enumerate(optimizer_ft.param_groups):
    print(f"  Group {i}: LR = {param_group['lr']}")

### 4.3 Train Fine-Tuned Model

In [ ]:
# Track metrics
ft_train_losses = []
ft_train_accs = []
ft_test_losses = []
ft_test_accs = []

print("Training Fine-Tuned Model...\n")
start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, train_acc = train_one_epoch(
        finetuned_model, train_loader, criterion, optimizer_ft, device
    )
    ft_train_losses.append(train_loss)
    ft_train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, _, _ = evaluate(
        finetuned_model, test_loader, criterion, device
    )
    ft_test_losses.append(test_loss)
    ft_test_accs.append(test_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    # Step scheduler
    scheduler_ft.step()
    print(f"Learning rates: {[group['lr'] for group in optimizer_ft.param_groups]}")

ft_time = time.time() - start_time
print(f"\nFine-tuning completed in {ft_time:.2f} seconds")
print(f"Best test accuracy: {max(ft_test_accs):.2f}%")

## Part 5: Strategy 3 - Training from Scratch

For comparison, let's train ResNet18 from scratch (random initialization).

In [ ]:
# Create model from scratch (no pretrained weights)
scratch_model = models.resnet18(pretrained=False)
scratch_model.fc = nn.Linear(scratch_model.fc.in_features, 10)
scratch_model = scratch_model.to(device)

# All parameters are trainable
trainable_params = sum(p.numel() for p in scratch_model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

# Optimizer with higher learning rate (no pretrained weights to preserve)
optimizer_scratch = optim.Adam(scratch_model.parameters(), lr=0.001)
scheduler_scratch = optim.lr_scheduler.StepLR(optimizer_scratch, step_size=5, gamma=0.1)

In [ ]:
# Track metrics
scratch_train_losses = []
scratch_train_accs = []
scratch_test_losses = []
scratch_test_accs = []

print("Training from Scratch...\n")
start_time = time.time()

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, train_acc = train_one_epoch(
        scratch_model, train_loader, criterion, optimizer_scratch, device
    )
    scratch_train_losses.append(train_loss)
    scratch_train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, _, _ = evaluate(
        scratch_model, test_loader, criterion, device
    )
    scratch_test_losses.append(test_loss)
    scratch_test_accs.append(test_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    scheduler_scratch.step()

scratch_time = time.time() - start_time
print(f"\nTraining from scratch completed in {scratch_time:.2f} seconds")
print(f"Best test accuracy: {max(scratch_test_accs):.2f}%")

## Part 6: Compare Different Architectures

Let's try different pretrained architectures: VGG16 and MobileNetV2

In [ ]:
def train_pretrained_model(model_name, num_epochs=10):
    """Train a pretrained model with feature extraction."""
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}\n")
    
    # Load model
    if model_name == 'VGG16':
        model = models.vgg16(pretrained=True)
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, 10)
        # Freeze all except classifier
        for param in model.features.parameters():
            param.requires_grad = False
            
    elif model_name == 'MobileNetV2':
        model = models.mobilenet_v2(pretrained=True)
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, 10)
        # Freeze all except classifier
        for param in model.features.parameters():
            param.requires_grad = False
    
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    model = model.to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}\n")
    
    # Setup training
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    # Track metrics
    train_accs = []
    test_accs = []
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Train
        _, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        train_accs.append(train_acc)
        
        # Evaluate
        _, test_acc, _, _ = evaluate(model, test_loader, criterion, device)
        test_accs.append(test_acc)
        
        print(f"Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%\n")
    
    training_time = time.time() - start_time
    
    return {
        'model': model,
        'train_accs': train_accs,
        'test_accs': test_accs,
        'best_acc': max(test_accs),
        'time': training_time,
        'total_params': total_params,
        'trainable_params': trainable_params
    }

# Train VGG16
vgg_results = train_pretrained_model('VGG16', num_epochs=5)

# Train MobileNetV2
mobilenet_results = train_pretrained_model('MobileNetV2', num_epochs=5)

## Part 7: Comprehensive Comparison and Analysis

### 7.1 Training Curves Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Training accuracy
axes[0, 0].plot(fe_train_accs, label='Feature Extraction', marker='o')
axes[0, 0].plot(ft_train_accs, label='Fine-Tuning', marker='s')
axes[0, 0].plot(scratch_train_accs, label='From Scratch', marker='^')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy (%)')
axes[0, 0].set_title('Training Accuracy Comparison (ResNet18)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Test accuracy
axes[0, 1].plot(fe_test_accs, label='Feature Extraction', marker='o')
axes[0, 1].plot(ft_test_accs, label='Fine-Tuning', marker='s')
axes[0, 1].plot(scratch_test_accs, label='From Scratch', marker='^')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].set_title('Test Accuracy Comparison (ResNet18)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Training loss
axes[1, 0].plot(fe_train_losses, label='Feature Extraction', marker='o')
axes[1, 0].plot(ft_train_losses, label='Fine-Tuning', marker='s')
axes[1, 0].plot(scratch_train_losses, label='From Scratch', marker='^')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].set_title('Training Loss Comparison (ResNet18)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Test loss
axes[1, 1].plot(fe_test_losses, label='Feature Extraction', marker='o')
axes[1, 1].plot(ft_test_losses, label='Fine-Tuning', marker='s')
axes[1, 1].plot(scratch_test_losses, label='From Scratch', marker='^')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].set_title('Test Loss Comparison (ResNet18)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 7.2 Architecture Comparison

In [ ]:
# Compare different architectures
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Test accuracy over epochs
ax1.plot(fe_test_accs, label='ResNet18', marker='o')
ax1.plot(vgg_results['test_accs'], label='VGG16', marker='s')
ax1.plot(mobilenet_results['test_accs'], label='MobileNetV2', marker='^')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Test Accuracy (%)')
ax1.set_title('Architecture Comparison (Feature Extraction)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bar chart of best accuracies
architectures = ['ResNet18', 'VGG16', 'MobileNetV2']
best_accs = [max(fe_test_accs), vgg_results['best_acc'], mobilenet_results['best_acc']]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

bars = ax2.bar(architectures, best_accs, color=colors, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Best Test Accuracy (%)')
ax2.set_title('Best Test Accuracy by Architecture')
ax2.set_ylim([min(best_accs) - 5, 100])
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, best_accs):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### 7.3 Summary Table

In [ ]:
import pandas as pd

# Create summary dataframe
summary_data = [
    {
        'Strategy': 'Feature Extraction',
        'Architecture': 'ResNet18',
        'Best Test Acc (%)': max(fe_test_accs),
        'Training Time (s)': fe_time,
        'Trainable Params': sum(p.numel() for p in feature_extractor.parameters() if p.requires_grad),
        'Total Params': sum(p.numel() for p in feature_extractor.parameters())
    },
    {
        'Strategy': 'Fine-Tuning',
        'Architecture': 'ResNet18',
        'Best Test Acc (%)': max(ft_test_accs),
        'Training Time (s)': ft_time,
        'Trainable Params': sum(p.numel() for p in finetuned_model.parameters() if p.requires_grad),
        'Total Params': sum(p.numel() for p in finetuned_model.parameters())
    },
    {
        'Strategy': 'From Scratch',
        'Architecture': 'ResNet18',
        'Best Test Acc (%)': max(scratch_test_accs),
        'Training Time (s)': scratch_time,
        'Trainable Params': sum(p.numel() for p in scratch_model.parameters() if p.requires_grad),
        'Total Params': sum(p.numel() for p in scratch_model.parameters())
    },
    {
        'Strategy': 'Feature Extraction',
        'Architecture': 'VGG16',
        'Best Test Acc (%)': vgg_results['best_acc'],
        'Training Time (s)': vgg_results['time'],
        'Trainable Params': vgg_results['trainable_params'],
        'Total Params': vgg_results['total_params']
    },
    {
        'Strategy': 'Feature Extraction',
        'Architecture': 'MobileNetV2',
        'Best Test Acc (%)': mobilenet_results['best_acc'],
        'Training Time (s)': mobilenet_results['time'],
        'Trainable Params': mobilenet_results['trainable_params'],
        'Total Params': mobilenet_results['total_params']
    }
]

df_summary = pd.DataFrame(summary_data)
df_summary['Param Ratio (%)'] = 100 * df_summary['Trainable Params'] / df_summary['Total Params']

print("\n" + "="*100)
print("COMPREHENSIVE COMPARISON SUMMARY")
print("="*100)
print(df_summary.to_string(index=False))
print("="*100)

# Key insights
print("\nKEY INSIGHTS:")
print(f"1. Best overall accuracy: {df_summary['Best Test Acc (%)'].max():.2f}% "
      f"({df_summary.loc[df_summary['Best Test Acc (%)'].idxmax(), 'Strategy']} - "
      f"{df_summary.loc[df_summary['Best Test Acc (%)'].idxmax(), 'Architecture']})")
print(f"2. Fastest training: {df_summary['Training Time (s)'].min():.2f}s "
      f"({df_summary.loc[df_summary['Training Time (s)'].idxmin(), 'Strategy']} - "
      f"{df_summary.loc[df_summary['Training Time (s)'].idxmin(), 'Architecture']})")
print(f"3. Most parameter efficient: {df_summary['Param Ratio (%)'].min():.2f}% trainable "
      f"({df_summary.loc[df_summary['Param Ratio (%)'].idxmin(), 'Architecture']})")
print(f"4. Transfer learning benefit: +{max(fe_test_accs) - max(scratch_test_accs):.2f}% over training from scratch")

## Part 8: Detailed Evaluation of Best Model

Let's do a comprehensive evaluation of the best performing model.

In [ ]:
# Select best model (fine-tuned ResNet18)
best_model = finetuned_model
print("Evaluating Fine-Tuned ResNet18 (Best Model)\n")

# Get predictions
_, test_acc, predictions, true_labels = evaluate(best_model, test_loader, criterion, device)
print(f"Final Test Accuracy: {test_acc:.2f}%\n")

### 8.1 Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix - Fine-Tuned ResNet18 on CIFAR-10', fontsize=14)
plt.tight_layout()
plt.show()

### 8.2 Per-Class Analysis

In [ ]:
# Classification report
print("Classification Report:\n")
print(classification_report(true_labels, predictions, target_names=classes))

# Per-class accuracy
per_class_acc = cm.diagonal() / cm.sum(axis=1)

# Visualize per-class accuracy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
bars = ax1.bar(classes, per_class_acc * 100, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_ylabel('Accuracy (%)', fontsize=12)
ax1.set_title('Per-Class Accuracy', fontsize=14)
ax1.set_ylim([0, 100])
ax1.grid(True, alpha=0.3, axis='y')
ax1.axhline(y=test_acc, color='red', linestyle='--', label=f'Overall: {test_acc:.2f}%')
ax1.legend()

# Add value labels
for bar, acc in zip(bars, per_class_acc * 100):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{acc:.1f}%', ha='center', va='bottom', fontsize=9)

# Sorted by accuracy
sorted_indices = np.argsort(per_class_acc)
sorted_classes = [classes[i] for i in sorted_indices]
sorted_accs = per_class_acc[sorted_indices] * 100

bars2 = ax2.barh(sorted_classes, sorted_accs, color='lightcoral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Accuracy (%)', fontsize=12)
ax2.set_title('Per-Class Accuracy (Sorted)', fontsize=14)
ax2.set_xlim([0, 100])
ax2.grid(True, alpha=0.3, axis='x')
ax2.axvline(x=test_acc, color='red', linestyle='--', label=f'Overall: {test_acc:.2f}%')
ax2.legend()

plt.tight_layout()
plt.show()

# Print hardest and easiest classes
print(f"\nEasiest class: {classes[per_class_acc.argmax()]} ({per_class_acc.max()*100:.2f}%)")
print(f"Hardest class: {classes[per_class_acc.argmin()]} ({per_class_acc.min()*100:.2f}%)")

### 8.3 Misclassification Analysis

In [ ]:
# Find most common misclassifications
cm_no_diag = cm.copy()
np.fill_diagonal(cm_no_diag, 0)

# Top 5 misclassifications
print("Top 5 Most Common Misclassifications:\n")
flat_indices = np.argsort(cm_no_diag.ravel())[::-1]
for i, flat_idx in enumerate(flat_indices[:5]):
    true_idx, pred_idx = np.unravel_index(flat_idx, cm.shape)
    count = cm_no_diag[true_idx, pred_idx]
    print(f"{i+1}. {classes[true_idx]:>12} → {classes[pred_idx]:>12}: {count:4d} errors")

### 8.4 Visualize Correct and Incorrect Predictions

In [ ]:
# Get some test images without normalization for visualization
test_dataset_viz = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, 
    transform=transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
)

# Find examples of correct and incorrect predictions
correct_indices = np.where(predictions == true_labels)[0]
incorrect_indices = np.where(predictions != true_labels)[0]

# Visualize correct predictions
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Correct Predictions', fontsize=14, y=1.02)

for i, ax in enumerate(axes.ravel()):
    idx = correct_indices[i]
    img, _ = test_dataset_viz[idx]
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(f'Predicted: {classes[predictions[idx]]}\nTrue: {classes[true_labels[idx]]}', 
                 fontsize=10, color='green')
    ax.axis('off')

plt.tight_layout()
plt.show()

# Visualize incorrect predictions
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Incorrect Predictions', fontsize=14, y=1.02)

for i, ax in enumerate(axes.ravel()):
    idx = incorrect_indices[i]
    img, _ = test_dataset_viz[idx]
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(f'Predicted: {classes[predictions[idx]]}\nTrue: {classes[true_labels[idx]]}', 
                 fontsize=10, color='red')
    ax.axis('off')

plt.tight_layout()
plt.show()

## Part 9: Key Takeaways and Best Practices

### 9.1 When to Use Each Strategy

**Feature Extraction (Frozen Backbone):**
- ✅ Use when: Small dataset, similar to ImageNet
- ✅ Advantages: Fast training, less data required, no overfitting
- ❌ Disadvantages: May not adapt well to very different domains
- 📊 Our result: ~{max(fe_test_accs):.1f}% accuracy

**Fine-Tuning:**
- ✅ Use when: Medium dataset, somewhat different from ImageNet
- ✅ Advantages: Better accuracy, adapts to new domain
- ❌ Disadvantages: Slower, requires careful learning rate tuning
- 📊 Our result: ~{max(ft_test_accs):.1f}% accuracy

**Training from Scratch:**
- ✅ Use when: Very large dataset, very different from ImageNet
- ✅ Advantages: Complete control, no domain mismatch
- ❌ Disadvantages: Requires massive data, slow convergence
- 📊 Our result: ~{max(scratch_test_accs):.1f}% accuracy

### 9.2 Best Practices

1. **Always start with transfer learning** unless you have >1M samples
2. **Use discriminative learning rates** when fine-tuning
3. **Match preprocessing** to pretrained model (ImageNet normalization)
4. **Freeze batch norm layers** when fine-tuning on small batches
5. **Try multiple architectures** - different models excel at different tasks
6. **Progressive unfreezing** - unfreeze layers gradually if needed

### 9.3 Architecture Selection Guidelines

**ResNet:**
- Good all-around choice
- Skip connections help with gradient flow
- Multiple sizes available (18, 34, 50, 101, 152)

**VGG:**
- Simple architecture, easy to understand
- Large model size, slower training
- Good for feature extraction

**MobileNet:**
- Designed for mobile/edge devices
- Much smaller, faster
- Slight accuracy trade-off
- Great for deployment

### 9.4 Mathematical Insight: Why Transfer Learning Works

From an optimization perspective, pretrained weights provide a good initialization:

$$
\theta^* = \arg\min_{\theta} L(\theta; D_{\text{target}})
$$

Starting from $\theta_{\text{pretrained}}$ (learned on ImageNet) is much closer to $\theta^*$ than random initialization, especially for early layers that learn universal features.

This reduces the optimization path length and prevents overfitting on small datasets.

## Part 10: (Optional) Advanced Topics

### Progressive Unfreezing

For very different domains, try progressive unfreezing:
1. Train only fc layer for N epochs
2. Unfreeze layer4, train for N epochs
3. Unfreeze layer3, train for N epochs
4. Continue as needed

This prevents catastrophic forgetting.

### Domain-Specific Augmentation

CIFAR-10 benefits from:
- Horizontal flips (makes sense for natural images)
- Random crops
- Color jittering

But NOT:
- Vertical flips (airplanes don't fly upside down)
- Large rotations (cars aren't rotated 90°)

Always consider domain knowledge when choosing augmentations.

---

## Summary

Congratulations! You've mastered transfer learning and pretrained models. You now understand:

✅ Why transfer learning works (feature hierarchy)  
✅ Feature extraction vs fine-tuning strategies  
✅ Working with multiple architectures (ResNet, VGG, MobileNet)  
✅ Discriminative learning rates and layer freezing  
✅ When to use each strategy  
✅ Comprehensive model evaluation  

**Achievement:** >85% accuracy on CIFAR-10 using transfer learning!

**Key Result:** Transfer learning provided +{max(fe_test_accs) - max(scratch_test_accs):.1f}% improvement over training from scratch

**Time spent:** ~3-4 hours

**Next:** Day 7 - Model Evaluation and Metrics (comprehensive evaluation beyond accuracy)